In [ ]:
import  pandas as  pd

In [ ]:
dataset = pd.read_csv('D:/test.csv')

df = pd.DataFrame(dataset) 
df.head()

In [ ]:
import  matplotlib.pyplot as plt

In [ ]:
plt.hist(df[0:30].F2  , bins='auto'  ,color='r',density=True);
plt.hist(df[31: ].F2 , bins='auto'  ,color='b',density=True);

from matplotlib.patches import Rectangle
cc = plt.get_cmap('jet')
h = [Rectangle((0,0),1,1,color=c,ec="k") for c in [cc(0.85),cc(0.25)]]

plt.legend(h , ["class1","class2"]);

In [ ]:
C1   = df[0:30].F3
C2   = df[31: ].F3
data =[C1 ,C2]

In [ ]:
plt.boxplot(data, notch=True , showfliers=False );     #, patch_artist=True

In [ ]:
fig, ax1 = plt.subplots(figsize=(8, 5))
fig.subplots_adjust(left=0.5, right=0.9, top=1, bottom=0.25)

bp = plt.boxplot(data, patch_artist=True, notch=True, vert=1, whis=2) #,  sym='w'

plt.setp( bp['boxes']    , color='b')
plt.setp( bp['whiskers'] , color='g')
plt.setp( bp['fliers']   , color='r', marker='+')


ax1.set_title('F2')
ax1.set_ylabel('Value')
ax1.yaxis.grid(True, linestyle='-', which='major', color='lightgrey', alpha=0.5)


In [ ]:
import seaborn as s
ax = s.boxplot(data=data , linewidth=2 , notch=True , palette="Set1" ) 
#ax = s.swarmplot(data=data, color="0.25")

In [ ]:
dfd = df.drop( ['F1'] , axis=1 )

In [ ]:
X = dfd.drop( ['target'] , axis = 1 )
y = dfd['target'] 

In [ ]:
from  sklearn.linear_model      import  LogisticRegression
from  sklearn.neural_network    import  MLPClassifier
from  sklearn                   import  svm
from  sklearn.svm               import  SVC
from  sklearn.tree              import  DecisionTreeClassifier

In [ ]:
LR  = LogisticRegression(solver='lbfgs' , random_state=0 , C=0.3, penalty='l2')
#MLP = MLPClassifier(solver='lbfgs', hidden_layer_sizes=(1) , alpha=1e-5, random_state=1) 
#SVM = SVC(C=1, kernel = 'rbf', random_state = 0, gamma='scale')    
#SVM = SVC(kernel='linear', C=1)  
#SVM = SVC(kernel='poly', degree=2, C=0.5, gamma='scale')   
#DT  = DecisionTreeClassifier(criterion='gini', max_depth=3, random_state=0)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.3, random_state=0)

In [ ]:
Xtrain.shape

In [ ]:
ytest.shape

In [ ]:
ytest

In [ ]:
from sklearn.preprocessing import StandardScaler  
scaler = StandardScaler()  
scaler.fit(Xtrain)  
Xtrain = scaler.transform(Xtrain)  
Xtest  = scaler.transform(Xtest) 

In [ ]:
LR.fit (Xtrain , ytrain);
#MLP.fit(Xtrain , ytrain) 
#SVM.fit(Xtrain , ytrain)
#DT.fit (Xtrain , ytrain)


In [ ]:
ypLR   = LR.predict(Xtest)
#ypMLP  = MLP.predict(Xtest)
#ypSVM  = SVM.predict(Xtest)
#ypDT   = DT.predict(Xtest)

In [ ]:
from  sklearn.metrics   import  classification_report

In [ ]:
print(classification_report(ytest, ypLR , target_names=['CLASS2', 'CLASS1'] ))

In [ ]:
from  sklearn.metrics   import  confusion_matrix

In [ ]:
confusion_matrix(ytest, ypLR)

In [ ]:
tn, fp, fn, tp = confusion_matrix(ytest, ypLR).ravel()
tn, fp, fn, tp

In [ ]:
LR.score(Xtrain , ytrain)   
#SVM.score(Xtrain , ytrain)   
#MLP.score(Xtrain , ytrain)   
#DT.score (Xtrain , ytrain)  

In [ ]:
LR.score(Xtest , ytest)    #accuracy_score(ytest, ypLR)
#MLP.score(Xtest , ytest)  
#SVM.score(Xtest , ytest)    
#DT.score (Xtest , ytest)   

In [ ]:
import  numpy  as  np

In [ ]:
from  sklearn.model_selection   import  cross_val_score

In [ ]:
np.mean(cross_val_score(LR , X , y , cv=10))    
#np.mean(cross_val_score(MLP , X , y , cv=10))    
#np.mean(cross_val_score(DT  , X , y , cv=10))    
#np.mean(cross_val_score(SVM , X , y , cv=10)) 

In [ ]:
from  sklearn.metrics  import  roc_curve, auc
from  sklearn.model_selection   import  StratifiedKFold
from  scipy  import  interp

In [ ]:
cv = StratifiedKFold(n_splits = 6)

tprs = []
aucs = []
mean_fpr = np.linspace(0, 1, 100)
i = 0

for train, test in cv.split(X, y):
    cf = LR.fit(Xtrain, ytrain)
    probas_ = cf.predict_proba(Xtest)
    fpr, tpr, thresholds = roc_curve(ytest, probas_[:, 1])
    tprs.append(interp(mean_fpr, fpr, tpr))
    tprs[-1][0] = 0.0
    roc_auc = auc(fpr, tpr)
    aucs.append(roc_auc)
    i += 1    
    
            
mean_tpr      = np.mean(tprs, axis=0)
mean_tpr[-1]  = 1.0
mean_auc      = auc(mean_fpr, mean_tpr)
std_auc       = np.std(aucs)
plt.plot(mean_fpr, mean_tpr, color='cyan',label=r'MLP' ,lw=2, alpha=.8)
std_tpr    = np.std(tprs, axis=0)
tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
tprs_lower = np.maximum(mean_tpr - std_tpr, 0)


plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')

plt.title('Mean ROC')
plt.legend(loc="lower right");

plt.savefig('D:/roc.png')